In [1]:
#imports 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats

PROJECT_PATH = "/home/igormago/PycharmProjects/doutorado"
NOTEBOOKS_CSV = PROJECT_PATH + "/notebooks/csv/"

matplotlib.style.use('ggplot') #style of plots

#to plot in this notebook
%matplotlib inline

#reading the dataframe
df1 = pd.read_csv("csv/list_matches.csv") 
df2 = pd.read_csv("csv/resumeMatchesByYear_avg.csv")
df3 = pd.read_csv("csv/resumeMatchesByYear_max.csv")


#legends
legendHDA = ['Home','Draw','Away']
legendFMU = ['Favorite','Medium','Underdog']

In [2]:
def setPct(df):

    df['PCT_H'] = df['CH'] / df['MP']
    df['PCT_D'] = df['CD'] / df['MP']
    df['PCT_A'] = df['CA'] / df['MP']
    df['PCT_F'] = df['CF'] / df['MO']
    df['PCT_M'] = df['CM'] / df['MO']
    df['PCT_U'] = df['CU'] / df['MO']
    
def sumCumulative(list):

    sum = 0
    cumulativeList = []
    for i in list:
        sum = sum + i
        cumulativeList.append(sum)

    return cumulativeList

def setCumulative(dfAvg, dfMax):
    dfAvg['CH_AVG'] = sumCumulative(dfAvg['PLH'].tolist())
    dfAvg['CD_AVG'] = sumCumulative(dfAvg['PLD'].tolist())
    dfAvg['CA_AVG'] = sumCumulative(dfAvg['PLA'].tolist())
    dfAvg['CF_AVG'] = sumCumulative(dfAvg['PLF'].tolist())
    dfAvg['CM_AVG'] = sumCumulative(dfAvg['PLM'].tolist())
    dfAvg['CU_AVG'] = sumCumulative(dfAvg['PLU'].tolist())
    
    dfAvg['CH_MAX'] = sumCumulative(dfMax['PLF'].tolist())
    dfAvg['CD_MAX'] = sumCumulative(dfMax['PLD'].tolist())
    dfAvg['CA_MAX'] = sumCumulative(dfMax['PLA'].tolist())
    dfAvg['CF_MAX'] = sumCumulative(dfMax['PLF'].tolist())
    dfAvg['CM_MAX'] = sumCumulative(dfMax['PLM'].tolist())
    dfAvg['CU_MAX'] = sumCumulative(dfMax['PLU'].tolist())

    
serieA = df2[df2['CHAMP'] == 'serie-a'].copy()
serieB = df2[df2['CHAMP'] == 'serie-b'].copy()

serieAmax = df3[df3['CHAMP'] == 'serie-a'].copy()
serieBmax = df3[df3['CHAMP'] == 'serie-b'].copy()

serieA.index = pd.to_datetime(serieA.YR,format='%Y')
serieB.index = pd.to_datetime(serieB.YR,format='%Y')

setPct(serieA)
setPct(serieB)
setCumulative(serieA,serieAmax)
setCumulative(serieB,serieBmax)



serieA.to_csv(NOTEBOOKS_CSV + 'serieA_matches.csv', index=False);
serieB.to_csv(NOTEBOOKS_CSV + 'serieB_matches.csv', index=False);

In [3]:
def calcResumeByYear(df):
    
    frame = pd.DataFrame()
   
    for g in range(0,8):

        frame['TH' + str(g)] = df['CM'][(df.HG == g) & (df.AG != g)].groupby(df.YR).sum()
        frame['TA' + str(g)] = df['CM'][(df.HG != g) & (df.AG == g)].groupby(df.YR).sum()
        frame['TD' + str(g)] = df['CM'][(df.HG == g) & (df.AG == g)].groupby(df.YR).sum()

        frame['RFH' + str(g)] = df['CM'][(df.RSL == 0) & (df.HG == g)].groupby(df.YR).sum()
        frame['RFD' + str(g)] = df['CM'][(df.RSL == 1) & (df.HG == g)].groupby(df.YR).sum()
        frame['RFA' + str(g)] = df['CM'][(df.RSL == 2) & (df.AG == g)].groupby(df.YR).sum()

        frame['RAH' + str(g)] = df['CM'][(df.RSL == 0) & (df.AG == g)].groupby(df.YR).sum()
        frame['RAD' + str(g)] = df['CM'][(df.RSL == 1) & (df.AG == g)].groupby(df.YR).sum()
        frame['RAA' + str(g)] = df['CM'][(df.RSL == 2) & (df.HG == g)].groupby(df.YR).sum()

        frame.fillna(0, inplace=True)
        frame['TM' + str(g)] = frame['TH' + str(g)] + frame['TA' + str(g)] + frame['TD' + str(g)]

        frame['PFT' + str(g)] = frame['RFD' + str(g)] + (frame['RFH' + str(g)] + frame['RFA' + str(g)]) * 3 
        frame['PAT' + str(g)] = frame['RAD' + str(g)] + (frame['RAH' + str(g)] + frame['RAA' + str(g)]) * 3 

        frame['PFH' + str(g)] = (frame['RFH' + str(g)] * 3) + frame['RFD' + str(g)]
        frame['PFA' + str(g)] = (frame['RFA' + str(g)] * 3) + frame['RFD' + str(g)]                      
        frame['PAH' + str(g)] = (frame['RAH' + str(g)] * 3) + frame['RAD' + str(g)]
        frame['PAA' + str(g)] = (frame['RAA' + str(g)] * 3) + frame['RAD' + str(g)]

        frame['PPFT'+ str(g)] = frame['PFT' + str(g)] / frame['TM' + str(g)]
        frame['PPAT'+ str(g)] = frame['PAT' + str(g)] / frame['TM' + str(g)]

        frame['PPFH'+ str(g)] = frame['PFH' + str(g)] / (frame['TH' + str(g)] + frame['TD' + str(g)])
        frame['PPFA'+ str(g)] = frame['PFA' + str(g)] / (frame['TA' + str(g)] + frame['TD' + str(g)])           
        frame['PPAH'+ str(g)] = frame['PAH' + str(g)] / (frame['TA' + str(g)] + frame['TD' + str(g)])
        frame['PPAA'+ str(g)] = frame['PAA' + str(g)] / (frame['TH' + str(g)] + frame['TD' + str(g)])
        
        frame.fillna(0, inplace=True)                                                                                        
                                                                                                 
        
    return frame

serieA = df1[df1['CHAMP'] == 'serie-a'].copy()
serieB = df1[df1['CHAMP'] == 'serie-b'].copy()

resumeA = serieA.groupby(['YR','HG','AG','RSL']).size()
resumeA = resumeA.to_frame('CM')
resumeA.reset_index(inplace=True)

resumeB = serieB.groupby(['YR','HG','AG','RSL']).size()
resumeB = resumeB.to_frame('CM')
resumeB.reset_index(inplace=True)

cumulativeA = calcResumeByYear(resumeA)
cumulativeB = calcResumeByYear(resumeB)

cumulativeA.reset_index(inplace=True)
cumulativeB.reset_index(inplace=True)

cumulativeA.to_csv(NOTEBOOKS_CSV + 'serieA_by_year.csv', index=False);
cumulativeB.to_csv(NOTEBOOKS_CSV + 'serieB_by_year.csv', index=False);

In [4]:
def calcResumeByGoals(df):
    
    frame = pd.DataFrame()

    frame['TH'] = df['CM'][(df.HG != df.AG)].groupby(df.HG).sum()
    frame['TA'] = df['CM'][(df.HG != df.AG)].groupby(df.AG).sum()
    frame['TD'] = df['CM'][(df.HG == df.AG)].groupby(df.HG).sum()
    
    frame['RFH'] = df['CM'][(df.HG != df.AG) & (df.RSL == 0)].groupby(df.HG).sum()
    frame['RFD'] = df['CM'][(df.HG == df.AG) & (df.RSL == 1)].groupby(df.HG).sum()
    frame['RFA'] = df['CM'][(df.HG != df.AG) & (df.RSL == 2)].groupby(df.AG).sum()
    
    frame['RAH'] = df['CM'][(df.HG != df.AG) & (df.RSL == 0)].groupby(df.AG).sum()
    frame['RAD'] = df['CM'][(df.HG == df.AG) & (df.RSL == 1)].groupby(df.AG).sum()
    frame['RAA'] = df['CM'][(df.HG != df.AG) & (df.RSL == 2)].groupby(df.HG).sum()
    
    frame.fillna(0, inplace=True)
    frame['TM'] = frame['TH'] + frame['TA'] + frame['TD']
    
    frame['PFT'] = frame['RFD'] + (frame['RFH'] + frame['RFA']) * 3 
    frame['PAT'] = frame['RAD'] + (frame['RAH'] + frame['RAA']) * 3

    frame['PFH'] = (frame['RFH'] * 3) + frame['RFD']
    frame['PFA'] = (frame['RFA'] * 3) + frame['RFD']
    frame['PAH'] = (frame['RAH'] * 3) + frame['RAD']
    frame['PAA'] = (frame['RAA'] * 3) + frame['RAD']
    
    frame['PPFT'] = frame['PFT'] / frame['TM']
    frame['PPAT'] = frame['PAT'] / frame['TM']
    
    frame['PPFH'] = frame['PFH'] / (frame['TH'] + frame['TD'])
    frame['PPFA'] = frame['PFA'] / (frame['TA'] + frame['TD'])
    frame['PPAH'] = frame['PAH'] / (frame['TA'] + frame['TD'])
    frame['PPAA'] = frame['PAA'] / (frame['TH'] + frame['TD'])
    
    frame.index.names = ['GOALS']
    return frame

serieA = df1[df1['CHAMP'] == 'serie-a'].copy()
serieB = df1[df1['CHAMP'] == 'serie-b'].copy()

resumeA = serieA.groupby(['YR','HG','AG','RSL']).size()
resumeA = resumeA.to_frame('CM')
resumeA.reset_index(inplace=True)

resumeB = serieB.groupby(['YR','HG','AG','RSL']).size()
resumeB = resumeB.to_frame('CM')
resumeB.reset_index(inplace=True)

cumulativeA = calcResumeByGoals(resumeA)
cumulativeB = calcResumeByGoals(resumeB)

cumulativeA.reset_index(inplace=True)
cumulativeB.reset_index(inplace=True)

cumulativeA.to_csv(NOTEBOOKS_CSV + 'serieA_by_goals.csv', index=False);
cumulativeB.to_csv(NOTEBOOKS_CSV + 'serieB_by_goals.csv', index=False);

In [5]:
def setBetResult(listResults, matchResult, betResult):
    if (matchResult == betResult):
        listResults.append(1)
    else:
        listResults.append(0)
        
def setBets(df,df2,local):
    betsPTA = []
    betsGFA = []
    betsGAA = []
    betsGD = []
    resultPTA = []
    resultGFA = []
    resultGAA = []
    resultGD = []

    for index, row in df2.iterrows():

        matchId = row['ID']
        teamHome = row['HT']
        teamAway = row['AT']
        result = row['RSL']

        if (local == True):
            tHome = df[(df.ID == matchId) & (df.TEAM == teamHome) & (df.LOC == 'H')]
            tAway = df[(df.ID == matchId) & (df.TEAM == teamAway) & (df.LOC == 'A')]
        else:
            tHome = df[(df.ID == matchId) & (df.TEAM == teamHome) & (df.LOC == 'B')]
            tAway = df[(df.ID == matchId) & (df.TEAM == teamAway) & (df.LOC == 'B')]

        ptaHome = tHome.ix[matchId,'PTA']
        ptaAway = tAway.ix[matchId,'PTA']
        gfaHome = tHome.ix[matchId,'GFA']
        gfaAway = tAway.ix[matchId,'GFA']
        gaaHome = tHome.ix[matchId,'GAA']
        gaaAway = tAway.ix[matchId,'GAA']
        gdHome = tHome.ix[matchId,'GD']
        gdAway = tAway.ix[matchId,'GD']
        
        if (ptaHome > ptaAway):
            betsPTA.append(0)
            setBetResult(resultPTA,result,0)
        elif (ptaHome < ptaAway):
            betsPTA.append(2)
            setBetResult(resultPTA,result,2)
        else:
            betsPTA.append(1)
            setBetResult(resultPTA,result,1)

        if (gfaHome > gfaAway):
            betsGFA.append(0)
            setBetResult(resultGFA,result,0)
        elif (gfaHome < gfaAway):
            betsGFA.append(2)
            setBetResult(resultGFA,result,2)
        else:
            betsGFA.append(1)
            setBetResult(resultGFA,result,1)

        if (gaaHome < gaaAway):
            betsGAA.append(0)
            setBetResult(resultGAA,result,0)
        elif (gaaHome > gaaAway):
            betsGAA.append(2)
            setBetResult(resultGAA,result,2)
        else:
            betsGAA.append(1)
            setBetResult(resultGAA,result,1)

        if (gdHome > gdAway):
            betsGD.append(0)
            setBetResult(resultGD,result,0)
        elif (gdHome < gdAway):
            betsGD.append(2)
            setBetResult(resultGD,result,2)
        else:
            betsGD.append(1)
            setBetResult(resultGD,result,1)
            
    df2['BPTA'] = betsPTA
    df2['BGFA'] = betsGFA
    df2['BGAA'] = betsGAA
    df2['BGB'] = betsGD
    df2['RPTA'] = resultPTA
    df2['RGFA'] = resultGFA
    df2['RGAA'] = resultGAA
    df2['RGD'] = resultGD
    
dfRank = pd.read_csv("csv/rankings.csv")
dfRank = dfRank[(dfRank.LMN == dfRank.MP)]
dfRank.index = dfRank.ID

dfMatch = pd.read_csv("csv/list_matches.csv")
setBets(dfRank, dfMatch, False)
dfMatch.to_csv(NOTEBOOKS_CSV + 'bets_total.csv', index=False);
setBets(dfRank, dfMatch, True)
dfMatch.to_csv(NOTEBOOKS_CSV + 'bets_local.csv', index=False);

In [6]:
df = pd.read_csv("csv/bets.csv")

frame = pd.DataFrame()

df['CHAMP'] = np.where(df['CHAMP'].str.contains('a'),'serie-a','serie-b')

grouped = df.ID[df.LOC == 0].groupby([df.CHAMP, df.YR, df.LMN]).count()
grouped2 = df.ID[(df.LOC == 0) & (df.HIT == True)].groupby([df.CHAMP, df.YR,df.LMN]).count()
grouped3 = df.RSLODD[(df.LOC == 0) & (df.HIT == True)].groupby([df.CHAMP, df.YR,df.LMN]).sum()


frame['MP'] = grouped
frame['HITS'] = grouped2
frame['AVG_HITS'] = frame['HITS'] / frame['MP']
frame['CUM_ODDS'] = grouped3
frame.reset_index(inplace=True)

frame.to_csv(NOTEBOOKS_CSV + 'resume_bets_total.csv', index=False);

In [7]:
df = pd.read_csv("csv/bets.csv")

frame = pd.DataFrame()

df['CHAMP'] = np.where(df['CHAMP'].str.contains('a'),'serie-a','serie-b')

grouped = df.ID[df.LOC == 1].groupby([df.CHAMP, df.YR, df.LMN]).count()
grouped2 = df.ID[(df.LOC == 1) & (df.HIT == True)].groupby([df.CHAMP, df.YR,df.LMN]).count()
grouped3 = df.RSLODD[(df.LOC == 1) & (df.HIT == True)].groupby([df.CHAMP, df.YR,df.LMN]).sum()

frame['MP'] = grouped
frame['HITS'] = grouped2
frame['AVG_HITS'] = frame['HITS'] / frame['MP']
frame['CUM_ODDS'] = grouped3
frame.reset_index(inplace=True)

frame.to_csv(NOTEBOOKS_CSV + 'resume_bets_local.csv', index=False);